In [24]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import density_app.functions.utilities as util
import density_app.functions.density_calc as calc

def get_my_data_no_file(date, cellname, temp, data, d1_res, d2_res, wavelength, optical_path):

    #collected_date, cell, tmp = get_info_from_fname(fp)
    #b_field, rot, r_err_MAE, r_err_SDT = get_processed_data_from_csv(fp)

    collected_date = date
    cell = cellname
    tmp = temp
    b_field = data["Magnetic Field (Gauss)"]
    rot = data["Rotation (Radians)"]
    #r_err_MAE = data["Rotation Mean Absolute Error"]
    r_err_STD = data["Rotation Standard Deviation"]

    fit_params, cov = util.linear_fit_data_with_error(b_field, rot,r_err_STD)
    slope = fit_params[0]
    #mae_avg_err = get_average_error(r_err_MAE)
    #std_avg_err = get_average_error(r_err_STD)
    cov_err = util.get_error_from_covar(cov)
    #max_err = np.array([mae_avg_err, std_avg_err, cov_err]).max()
    density = util.formatter(calc.rb_density(d1_res, d2_res, slope, optical_path, wavelength),4)
    density_error = util.formatter(calc.rb_density(d1_res, d2_res, cov_err, optical_path, wavelength),4)
    killian_val = util.ormatter(calc.killian_density(tmp),4)
    #create my data frame
    output = pd.DataFrame({'Date': [collected_date],
                        'Cell Name': [cell],
                        'Temperature':[tmp],
                        'Density':[density],
                        'Density Error':[density_error], 
                        'Killian Value':[killian_val], 
                        'D1 Resonance':  [d1_res],
                        'D2 Resonance':  [d2_res],
                                                    })
    return output


def get_info_from_fname(processed_filepath):
    #split the file path by / 
    fp_ary = str(processed_filepath).split('/')
    #take the last element of the array to get the file name
    #split that on . to separate extension from rest of name
    properties = fp_ary[-1].split('.')[0].split('_')
    #need to get the following params from the 
    col_date = properties[0]
    cell_name = properties[1].split('-')[1]
    temperature = properties[2].split('-')[1]
    return col_date, cell_name, temperature

def get_processed_data(filepath):
    data = pd.read_csv(filepath)
    x_data = data["Magnetic Field (Gauss)"].to_numpy()
    y_data = data["Rotation (Radians)"].to_numpy()
    err_STD = data["Rotation Standard Deviation"].to_numpy()
    return x_data, y_data, err_STD
		


#returns a linear fit for the specified data
#this version uses error values provided for the rotation values
def linear_fit_data(x_vals, y_vals, errors):
    # line for fitting
    def line(x, m):
        return x*m
    param, param_cov = curve_fit(line, x_vals, y_vals, sigma=errors, absolute_sigma=True)
    return param, param_cov

def get_density_error(d1, d2, slope, slope_error, probe_beam, probe_error, olen, olen_error):
    dtheta = (slope_error/slope)**2
    dprobe = (probe_error/probe_beam)**2
    dolen = (olen_error/olen)**2
    err = dtheta+dprobe+dolen
    print(err)
    rb_err = calc.rb_density(d1, d2, err, olen, probe_beam)
    return rb_err

f = "Data/Density/2024-08-12/2024-08-12_cell-314B_temp-130_trial-1_processed.csv"

x, y, err = get_processed_data(f)

slope, vars = linear_fit_data(x,y,err)
err_slope = np.sqrt(vars[0][0])
#print(slope[0])
#pint(vars[0][0])

probe = 0.0000780505
probe_err = 0.005e-7
opt = 3.7
opt_err = 0

rberr = get_density_error(calc.d1_resonance_lambda, calc.d2_resonance_lambda, slope[0], err_slope, probe, probe_err,opt, opt_err)
print(util.formatter(rberr,4))

rb = calc.rb_density(calc.d1_resonance_lambda, calc.d2_resonance_lambda, slope[0], opt, probe)
print(util.formatter(rb,4))
#final_data = pd.DataFrame({'Date': [],
#                        'Cell Name': [],
#                        'Temperature':[],
#                        'Density':[],
#                        'Density Error':[], 
#                        'Killian Value':[] })


2.558081446257033e-07
8.0263e+11
1.2228e+14
